In [12]:
from utils import model_utils

from pickle import load
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.models import load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from numpy import argmax

In [2]:
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

In [3]:
def generate_desc(model, tokenizer, photo, max_length):
    in_text = 'startseq'
    for i in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = model.predict([photo,sequence], verbose=0)
        yhat = argmax(yhat)
        word = word_for_id(yhat, tokenizer)
        if word is None:
            break
        in_text += ' ' + word
        if word == 'endseq':
            break
    return in_text

In [4]:
def initiate_encoder(arch='capsnet'):
    if arch=='capsnet':
        encoder_model = model_utils.load_DeepCapsNet(input_shape=(64,64,3), n_class=10, routings=3, \
                        weights=r'D:\CapsuleNetwork_ImageCaptioning\Flickr8k_image_captioning_using_CapsNet\weights\deep_caps_best_weights.h5')
    else:
        encoder_model = model_utils.load_VGG()
    return encoder_model

In [5]:
def extract_feature(model, arch, image_path):
    feature = None
    model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
    target_size = (64,64) if arch=='capsnet' else (224,224)
    try:
        image = load_img(image_path, target_size=target_size)
    except Exception as e:
        print('{} could not be opened. Skipping\n {}'.format(image_path,e))
        return None
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    if arch=='capsnet':
        feature = model.predict(image, verbose=0).reshape(-1, 10*32)
    else:
        image = preprocess_input(image)
        feature = model.predict(image, verbose=0)
    return feature

In [20]:
# encoder_model = initiate_encoder(arch='capsnet')
photo_feature = extract_feature(encoder_model, 'capsnet', r'C:\Users\jayde\OneDrive\Desktop\man.jpg')
max_length = 34

In [21]:
tokenizer = load(open('tokenizer.pkl', 'rb'))
test_model = load_model(r'weights\decoder_weights_ep60.h5')
print(generate_desc(test_model, tokenizer, photo_feature, max_length))

startseq hiviz while baby the wind background one sits horse the flames the bags the ready rocky the snowboarders violin thumbs the book player white stands jumps the the position the both horse the the
